In [1]:
from typing import Optional
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(default=None, description="The hair color of the person")
    height_in_meters: Optional[str] = Field(
        default=None, description="The height of the person in meters"
    )


In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm"
            "Only extract revelant information from the text"
            "If you do not know the value of an attribute asked to extract"
            "return null for the attribute's value",
        ),
        ("human", "{text}"),
    ]
)

In [6]:
import getpass, os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Please enter your Google API key: ")

In [7]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

In [8]:
structured_llm = llm.with_structured_output(schema=Person)

In [9]:
text = "Alan Smith is 6 feet tall and has brown hair"
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='Alan Smith', hair_color='brown', height_in_meters='1.83')

Multiple entities

In [10]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, 
        description="The hair color of the person"
        )
    height_in_meters: Optional[str] = Field(
        default=None, 
        description="The height of the person in meters"
    )

class Data(BaseModel):
    people: List[Person]


In [11]:
structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and I am 6 feet tall. Anna has the same color hair as me"
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(people=[Person(name='Jeff', hair_color='black', height_in_meters='1.83'), Person(name='Anna', hair_color='black', height_in_meters=None)])